## Import dependancies


In [1]:
import os
import sys

src_path = os.path.abspath(os.path.join("../src"))
if src_path not in sys.path:
    sys.path.append(src_path)

import data.make_dataset as make_dataset
import visualization.visualize as visualize
import features.build_features as build_features

# numpy and pandas for data manipulation
import numpy as np
import pandas as pd

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

DRAW_PLOTS = False

## Prepare data loading : set correct variable types


In [2]:
# Read column names
column_names = pd.read_csv(
    "../data/processed/application_train.csv", nrows=0
).columns.values

# Set column types according to fields description (https://static.openfoodfacts.org/data/data-fields.txt)
column_types = {
    col: "category"
    for col in column_names
    if col.startswith(("NAME_",))
    or col.endswith(("_TYPE"))
    or col
    in [
        "CODE_GENDER",
        "WEEKDAY_APPR_PROCESS_START",
        "FONDKAPREMONT_MODE",
        "HOUSETYPE_MODE",
        "WALLSMATERIAL_MODE",
        "EMERGENCYSTATE_MODE",
    ]
}
column_types |= {
    col: bool
    for col in column_names
    if col.startswith(("FLAG_", "REG_", "LIVE_"))
}


## Application and Test data loading and first observations


### Application training data


In [3]:
# Load application data
app_train_df = pd.read_csv(
    "../data/processed/application_train.csv",
    dtype=column_types,
    true_values=["Y", "Yes", "1"],
    false_values=["N", "No", "0"],
    na_values=["XNA"],
)

# Application data features
app_train_df.head()


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,False,True,0,202500.0,406597.5,24700.5,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,False,False,0,270000.0,1293502.5,35698.5,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,True,True,0,67500.0,135000.0,6750.0,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,False,True,0,135000.0,312682.5,29686.5,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,False,True,0,121500.0,513000.0,21865.5,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Application data columns info
app_train_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: bool(34), category(14), float64(65), int64(9)
memory usage: 187.7 MB


In [5]:
# Application data variables description
app_train_df.describe(include="all")


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511,307507,307511,307511,307511.000000,3.075110e+05,3.075110e+05,307499.000000,...,307511,307511,307511,307511,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
unique,NaN,NaN,2,2,2,2,NaN,NaN,NaN,NaN,...,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,Cash loans,F,False,True,NaN,NaN,NaN,NaN,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,278232,202448,202924,213312,NaN,NaN,NaN,NaN,...,305011,307328,307355,307408,NaN,NaN,NaN,NaN,NaN,NaN
mean,278180.518577,0.080729,NaN,NaN,NaN,NaN,0.417052,1.687979e+05,5.990260e+05,27108.573909,...,NaN,NaN,NaN,NaN,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,NaN,NaN,NaN,NaN,0.722121,2.371231e+05,4.024908e+05,14493.737315,...,NaN,NaN,NaN,NaN,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,2.565000e+04,4.500000e+04,1615.500000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,NaN,NaN,NaN,NaN,0.000000,1.125000e+05,2.700000e+05,16524.000000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,1.471500e+05,5.135310e+05,24903.000000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,NaN,NaN,NaN,NaN,1.000000,2.025000e+05,8.086500e+05,34596.000000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000


### Application testing data


In [6]:
# Load test data
app_test_df = pd.read_csv(
    "../data/processed/application_test.csv",
    dtype=column_types,
    true_values=["Y", "Yes", "1"],
    false_values=["N", "No", "0"],
    na_values=["XNA"],
)

# Test data features
app_test_df.head()


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,False,True,0,135000.0,568800.0,20560.5,450000.0,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,False,True,0,99000.0,222768.0,17370.0,180000.0,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,True,True,0,202500.0,663264.0,69777.0,630000.0,...,False,False,False,False,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,False,True,2,315000.0,1575000.0,49018.5,1575000.0,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,True,False,1,180000.0,625500.0,32067.0,625500.0,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Test data columns info
app_test_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 121 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: bool(34), category(14), float64(65), int64(8)
memory usage: 29.4 MB


In [8]:
# Test data variables description
app_test_df.describe(include="all")


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,48744.000000,48744,48744,48744,48744,48744.000000,4.874400e+04,4.874400e+04,48720.000000,4.874400e+04,...,48744,48744,48744,48744,42695.000000,42695.000000,42695.000000,42695.000000,42695.000000,42695.000000
unique,NaN,2,2,2,2,NaN,NaN,NaN,NaN,NaN,...,2,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Cash loans,F,False,True,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,48305,32678,32311,33658,NaN,NaN,NaN,NaN,NaN,...,48668,48744,48744,48744,NaN,NaN,NaN,NaN,NaN,NaN
mean,277796.676350,NaN,NaN,NaN,NaN,0.397054,1.784318e+05,5.167404e+05,29426.240209,4.626188e+05,...,NaN,NaN,NaN,NaN,0.002108,0.001803,0.002787,0.009299,0.546902,1.983769
std,103169.547296,NaN,NaN,NaN,NaN,0.709047,1.015226e+05,3.653970e+05,16016.368315,3.367102e+05,...,NaN,NaN,NaN,NaN,0.046373,0.046132,0.054037,0.110924,0.693305,1.838873
min,100001.000000,NaN,NaN,NaN,NaN,0.000000,2.694150e+04,4.500000e+04,2295.000000,4.500000e+04,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,188557.750000,NaN,NaN,NaN,NaN,0.000000,1.125000e+05,2.606400e+05,17973.000000,2.250000e+05,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,277549.000000,NaN,NaN,NaN,NaN,0.000000,1.575000e+05,4.500000e+05,26199.000000,3.960000e+05,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,367555.500000,NaN,NaN,NaN,NaN,1.000000,2.250000e+05,6.750000e+05,37390.500000,6.300000e+05,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000


## Exploratory Data Analysis (EDA)


### Look for empty columns

In [9]:
# Plot application columns emptiness ratio
if DRAW_PLOTS:
    visualize.plot_empty_values(app_train_df)


In [10]:
# Plot test columns emptiness ratio
if DRAW_PLOTS:
    visualize.plot_empty_values(app_test_df)


### Look at numerical variables


In [11]:
clean_app_train_df = build_features.drop_impossible_values(
    app_train_df,
    constraints={
        "AMT_INCOME_TOTAL": {"min": 0, "max": 20 * 1000 * 1000,},
        "DAYS_EMPLOYED": {"min": -100 * 365, "max": 0,},
        "AMT_REQ_CREDIT_BUREAU_QRT": {"min": 0, "max": 20,},
        "OBS_30_CNT_SOCIAL_CIRCLE": {"min": 0, "max": 200,},
        "DEF_30_CNT_SOCIAL_CIRCLE": {"min": 0, "max": 20,},
        "OBS_60_CNT_SOCIAL_CIRCLE": {"min": 0, "max": 200,},
        "DEF_60_CNT_SOCIAL_CIRCLE": {"min": 0, "max": 20,},
    },
)
clean_app_train_df = build_features.drop_outliers(
    clean_app_train_df, columns=["REGION_POPULATION_RELATIVE"],
)

# Draw the BoxPlots of each numeric column, split per Nutrition Grade
if DRAW_PLOTS:
    visualize.plot_boxes(
        dataframe=clean_app_train_df,
        plot_columns=[
            "AMT_INCOME_TOTAL",
            "AMT_CREDIT",
            "AMT_ANNUITY",
            "AMT_GOODS_PRICE",
            "DAYS_BIRTH",
            "DAYS_EMPLOYED",
            "OWN_CAR_AGE",
            "REGION_RATING_CLIENT",
            "REGION_RATING_CLIENT_W_CITY",
            "EXT_SOURCE_1",
            "EXT_SOURCE_2",
            "EXT_SOURCE_3",
            "DAYS_LAST_PHONE_CHANGE",
            "AMT_REQ_CREDIT_BUREAU_YEAR",
        ],
        categorical_column="TARGET",
    )


In [12]:
clean_app_test_df = build_features.drop_impossible_values(
    app_test_df,
    constraints={
        "DAYS_EMPLOYED": {"min": -100 * 365, "max": 0,},
        "AMT_REQ_CREDIT_BUREAU_QRT": {"min": 0, "max": 20,},
        "OBS_30_CNT_SOCIAL_CIRCLE": {"min": 0, "max": 200,},
        "DEF_30_CNT_SOCIAL_CIRCLE": {"min": 0, "max": 20,},
        "OBS_60_CNT_SOCIAL_CIRCLE": {"min": 0, "max": 200,},
        "DEF_60_CNT_SOCIAL_CIRCLE": {"min": 0, "max": 20,},
    },
)
clean_app_test_df = build_features.drop_outliers(
    clean_app_test_df, columns=["REGION_POPULATION_RELATIVE"],
)


### Look at categorical variables


In [13]:
if DRAW_PLOTS:
    visualize.plot_categories_bars(
        clean_app_train_df,
        plot_columns=[
            "NAME_CONTRACT_TYPE",
            "CODE_GENDER",
            "FLAG_OWN_CAR",
            "FLAG_OWN_REALTY",
            "NAME_INCOME_TYPE",
            "NAME_EDUCATION_TYPE",
            "NAME_FAMILY_STATUS",
            "NAME_HOUSING_TYPE",
            "OCCUPATION_TYPE",
            "FLAG_MOBIL",
        ],
        categorical_column="TARGET",
    )


#### One Hot Encoding

No ordinal data => One Hot Encoding is better than Label Encoding


In [14]:
# one-hot encoding of categorical variables
encoded_app_train_df = pd.get_dummies(clean_app_train_df, dtype=bool)
encoded_app_test_df = pd.get_dummies(clean_app_test_df, dtype=bool)

train_labels = encoded_app_train_df["TARGET"]

# Align the training and testing data, keep only columns present in both dataframes
encoded_app_train_df, encoded_app_test_df = encoded_app_train_df.align(
    encoded_app_test_df, join="inner", axis=1
)

# Add the target back in
encoded_app_train_df["TARGET"] = train_labels

print("Training Features shape: ", encoded_app_train_df.shape)
print("Testing Features shape: ", encoded_app_test_df.shape)


Training Features shape:  (217560, 240)
Testing Features shape:  (34522, 239)


### Missing values imputation


In [15]:
if os.path.exists("../data/processed/imputed_application_train.csv"):
    imputed_app_train_df = pd.read_csv(
        "../data/processed/imputed_application_train.csv", index_col=0
    )
else:
    imputed_app_train_df = build_features.impute_missing_values(
        encoded_app_train_df
    )


if os.path.exists("../data/processed/imputed_application_test.csv"):
    imputed_app_test_df = pd.read_csv(
        "../data/processed/imputed_application_test.csv", index_col=0
    )
else:
    imputed_app_test_df = build_features.impute_missing_values(
        encoded_app_test_df
    )


[IterativeImputer] Completing matrix with shape (217560, 240)


[IterativeImputer] Ending imputation round 1/10, elapsed time 106.82


[IterativeImputer] Change: 680165.776744538, scaled tolerance: 9000.0 


[IterativeImputer] Ending imputation round 2/10, elapsed time 217.34


[IterativeImputer] Change: 54281.31643709613, scaled tolerance: 9000.0 


[IterativeImputer] Ending imputation round 3/10, elapsed time 321.11


[IterativeImputer] Change: 56122.794531756546, scaled tolerance: 9000.0 


#### Features selection

Variables that are not highly correlated to an other, and at least a bit correlated to TARGET.

In [ ]:
corrs_app_train_df = imputed_app_train_df.corr()

if DRAW_PLOTS:
    fig = px.imshow(corrs_app_train_df,
        title="Correlations between features",
        width=1200,
        height=1200,
    )
    fig.show()

In [ ]:
simple_app_train_df = imputed_app_train_df.copy()
simple_app_test_df = imputed_app_test_df.copy()

highly_decorrelated_from_target_columns = []
for col in corrs_app_train_df.columns:
    if col != "TARGET" and (
        pd.isnull(corrs_app_train_df[col]["TARGET"])
        or abs(corrs_app_train_df[col]["TARGET"]) < 0.01
    ):
        highly_decorrelated_from_target_columns.append(col)
        print(
            "Highly de-correlated from TARGET : ",
            col,
            corrs_app_train_df[col]["TARGET"],
        )

simple_app_train_df.drop(
    columns=highly_decorrelated_from_target_columns,
    inplace=True,
    errors="ignore",
)
simple_app_test_df.drop(
    columns=highly_decorrelated_from_target_columns,
    inplace=True,
    errors="ignore",
)

simple_app_train_df.shape


In [ ]:
highly_correlated_columns = []
for i in range(len(corrs_app_train_df.columns)):
    for j in range(i + 1, len(corrs_app_train_df.columns)):
        if i != j and abs(corrs_app_train_df.iloc[i, j]) > 0.9:
            highly_correlated_columns.append(corrs_app_train_df.columns[i])
            print(
                "Highly correlated pair : ",
                corrs_app_train_df.columns[i],
                corrs_app_train_df.columns[j],
                corrs_app_train_df.iloc[i, j],
            )

simple_app_train_df.drop(
    columns=highly_correlated_columns, inplace=True, errors="ignore",
)
simple_app_test_df.drop(
    columns=highly_correlated_columns, inplace=True, errors="ignore",
)

simple_app_train_df.shape


In [ ]:
columns = list(corrs_app_train_df.columns)
columns.remove("TARGET")
top_correlated_to_target_columns = (
    pd.Series(data={col: corrs_app_train_df[col]["TARGET"] for col in columns})
    .map(abs)
    .sort_values(ascending=False)
    .head(20)
)

if DRAW_PLOTS:
    fig = px.bar(
        top_correlated_to_target_columns,
        color=top_correlated_to_target_columns.values,
        title="Top 20 Columns Correlated to Target",
        labels={
            "index": "Column name",
            "value": "Correlation",
            "color": "Correlation",
        },
        width=1200,
        height=800,
    )
    fig.show()


---
---
---
---
---

## 